<a href="https://colab.research.google.com/github/MuhammadNurilHuda/Materi/blob/main/mini-project/Payment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Klasifikasi apakah pembeli akan membeli motor credit atau cash**

*   Fitur yang digunakan = 'TYPE MOTOR', 'JENIS KELAMIN', 'STATUS RUMAH', 'PEKERJAAN', 'PENGELUARAN','PENDIDIKAN', 'SMH DIGUNAKAN UNTUK', 'YG MENGGUNAKAN SMH', 'HOBI'
*   besar dp, besar cicilan, lama cicilan akan diganti menjadi CICIL sebagai label




In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/MuhammadNurilHuda/Materi/main/dataset/konsumen.csv", sep=';')
df.head(4)

,SALES DATE,TYPE MOTOR,COLOR,KODE CUSTOMER,JENIS KELAMIN,TANGGAL LAHIR,KECAMATAN SURAT,KOTA SURAT,PROPINSI,STATUS RUMAH,JENIS PENJUALAN STNK,JENIS PENJUALAN SSU,BESAR DP,BESAR CICILAN,LAMA CICILAN,AGAMA,PEKERJAAN,PENGELUARAN,PENDIDIKAN,KEBERSEDIAAN DIHUBUNGI,MERK MOTOR SBLMNYA,TYPE MOTOR SBLMNYA,SMH DIGUNAKAN UNTUK,YG MENGGUNAKAN SMH,HOBI,KETERANGAN,KEWARGANEGARAAN
0,08/08/2019,REVO FIT FI,HITAM,INDIVIDUAL,LAKI-LAKI,25/08/1970,Manyar,Kab. Gresik,Jawa Timur,RUMAH SENDIRI,CASH,CASH,NaN,NaN,NaN,ISLAM,DOKTER,"RP 2.000.001,- S/D RP 3.000.000,-",AKADEMI/DIPLOMA,YES,HONDA,AT AUTOMATIC,BERDAGANG,SAYA SENDIRI,Adventure (Petualangan),NaN,WNI
1,16/10/2019,REVO FIT FI,HITAM,INDIVIDUAL,LAKI-LAKI,31/03/1972,Driyorejo,Kab. Gresik,Jawa Timur,RUMAH SENDIRI,CASH,CASH,NaN,NaN,NaN,ISLAM,"PEGAWAI SWASTA, INDUSTRI","RP 3.000.001,- S/D RP 4.000.000,-",SLTA/SMU,YES,HONDA,SPORT,BERDAGANG,SAYA SENDIRI,Adventure (Petualangan),NaN,WNI
2,07/10/2019,REVO FIT FI,HITAM,INDIVIDUAL,LAKI-LAKI,29/12/1998,Cerme,Kab. Gresik,Jawa Timur,RUMAH ORANG TUA / KELUARGA,CASH,CASH,NaN,NaN,NaN,ISLAM,MAHASISWA / PELAJAR,"RP 1.000.001,- S/D RP 1.500.000,-",SLTA/SMU,YES,HONDA,SPORT,BERDAGANG,SAYA SENDIRI,Adventure (Petualangan),NaN,WNI
3,09/09/2019,REVO FIT FI,HITAM,INDIVIDUAL,LAKI-LAKI,29/01/1983,Menganti,Kab. Gresik,Jawa Timur,RUMAH SENDIRI,CREDIT,CREDIT,1300000.0,634000.0,35.0,ISLAM,"PEGAWAI SWASTA, INDUSTRI","RP 1.000.001,- S/D RP 1.500.000,-",SLTA/SMU,YES,HONDA,AT AUTOMATIC,PEMAKAIAN JARAK DEKAT,SAYA SENDIRI,Makan,SOPAN,WNI


In [ ]:
# Menampilkan jumlah data null di setiap kolom
df.isnull().sum()

SALES DATE                    0
TYPE MOTOR                    0
COLOR                         0
KODE CUSTOMER                 0
JENIS KELAMIN                 0
TANGGAL LAHIR                 0
KECAMATAN SURAT               0
KOTA SURAT                    0
PROPINSI                      0
STATUS RUMAH                  0
JENIS PENJUALAN STNK          0
JENIS PENJUALAN SSU           0
BESAR DP                   7688
BESAR CICILAN              7688
LAMA CICILAN               7688
AGAMA                         0
PEKERJAAN                     0
PENGELUARAN                   0
PENDIDIKAN                    0
KEBERSEDIAAN DIHUBUNGI        0
MERK MOTOR SBLMNYA            0
TYPE MOTOR SBLMNYA            0
SMH DIGUNAKAN UNTUK           0
YG MENGGUNAKAN SMH            0
HOBI                          0
KETERANGAN                11354
KEWARGANEGARAAN               6
dtype: int64

In [ ]:
df.describe()

,BESAR DP,BESAR CICILAN,LAMA CICILAN
count,1.092800e+04,1.092800e+04,10928.000000
mean,4.041772e+06,9.614827e+05,29.467972
std,3.207775e+06,4.741369e+05,7.899691
min,1.000000e+00,3.000000e+01,2.000000
25%,2.100000e+06,7.310000e+05,23.000000
50%,3.000000e+06,8.490000e+05,35.000000
75%,5.000000e+06,1.083000e+06,35.000000
max,1.090000e+08,1.447000e+07,36.000000


In [ ]:
# memilih kolom yang digunakan
used_feature = ['TYPE MOTOR', 'JENIS KELAMIN', 'STATUS RUMAH', 'PEKERJAAN', 'PENGELUARAN','PENDIDIKAN', 'SMH DIGUNAKAN UNTUK', 'YG MENGGUNAKAN SMH', 'HOBI']

In [ ]:
# Menampilkan unique values dari setiap kolom yang digunakan
for i in used_feature:
    print(f"Unique value dari kolom {i} : {df[i].unique()}\n")

Unique value dari kolom TYPE MOTOR : ['REVO FIT FI' 'REVO CW FI' 'NEW SUPRA X 125 FI SW'
 'NEW SUPRA X 125 FI CW' 'BEAT POP CBS ISS' 'NEW VARIO 125 CBS ISS'
 'BEAT SPORTY CW' 'BEAT STREET CBS' 'BEAT SPORTY CBS ISS'
 'BEAT SPORTY CBS' 'ALL NEW SCOOPY' 'VARIO 125 CBS' 'VARIO 125 CBS ISS'
 'GENIO CBS' 'GENIO CBS ISS' 'ALL NEW SONIC 150 R' 'SUPRA GTR 150'
 'CB150 VERZA SW' 'CB150 VERZA CW' 'ALL NEW CBR150R' 'NEW CBR 150R STD'
 'NEW CBR 150R ABS' 'CB150R STREETFIRE STD' 'CB150R STREETFIRE SE'
 'CRF150L' 'PCX 150 CBS' 'PCX 150 ABS' 'PCX HYBRID' 'VARIO 150'
 'ADV150 CBS' 'ADV150 ABS' 'CBR 250RR STD' 'CBR 250RR ABS' 'FORZA'
 'SUPER CUB C125' 'NEW VARIO 150']

Unique value dari kolom JENIS KELAMIN : ['LAKI-LAKI' 'PEREMPUAN']

Unique value dari kolom STATUS RUMAH : ['RUMAH SENDIRI' 'RUMAH ORANG TUA / KELUARGA' 'RUMAH SEWA']

Unique value dari kolom PEKERJAAN : ['DOKTER' 'PEGAWAI SWASTA, INDUSTRI' 'MAHASISWA / PELAJAR'
 'WIRASWASTA / PEDAGANG, PERTANIAN / PERKEBUNAN / KEHUTANAN / PERIKANAN / PETE

## **Encode Categorical data**

---


> referensi : https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd


Menggunakan Label Encoder

In [ ]:
le = LabelEncoder()
df[used_feature] = df[used_feature].apply(le.fit_transform)
df[used_feature].head(3)

,TYPE MOTOR,JENIS KELAMIN,STATUS RUMAH,PEKERJAAN,PENGELUARAN,PENDIDIKAN,SMH DIGUNAKAN UNTUK,YG MENGGUNAKAN SMH,HOBI
0,30,0,1,0,4,0,1,3,0
1,30,0,1,9,5,4,1,3,0
2,30,0,0,4,2,4,1,3,0


Mengubah kolom besar dp menjadi kolom cicil dengan asumsi semua yang membayar dp pasti menyicil

In [ ]:
df = df.rename(columns={'BESAR DP':'CICIL'})

In [ ]:
df.head(1)

,SALES DATE,TYPE MOTOR,COLOR,KODE CUSTOMER,JENIS KELAMIN,TANGGAL LAHIR,KECAMATAN SURAT,KOTA SURAT,PROPINSI,STATUS RUMAH,JENIS PENJUALAN STNK,JENIS PENJUALAN SSU,CICIL,BESAR CICILAN,LAMA CICILAN,AGAMA,PEKERJAAN,PENGELUARAN,PENDIDIKAN,KEBERSEDIAAN DIHUBUNGI,MERK MOTOR SBLMNYA,TYPE MOTOR SBLMNYA,SMH DIGUNAKAN UNTUK,YG MENGGUNAKAN SMH,HOBI,KETERANGAN,KEWARGANEGARAAN
0,08/08/2019,30,HITAM,INDIVIDUAL,0,25/08/1970,Manyar,Kab. Gresik,Jawa Timur,1,CASH,CASH,NaN,NaN,NaN,ISLAM,0,4,0,YES,HONDA,AT AUTOMATIC,1,3,0,NaN,WNI


In [ ]:
# Menentukan cicil sebagai label
label=['CICIL']

In [ ]:
df[used_feature].isnull().sum()

TYPE MOTOR             0
JENIS KELAMIN          0
STATUS RUMAH           0
PEKERJAAN              0
PENGELUARAN            0
PENDIDIKAN             0
SMH DIGUNAKAN UNTUK    0
YG MENGGUNAKAN SMH     0
HOBI                   0
dtype: int64

In [ ]:
df[label].isnull().sum()

CICIL    7688
dtype: int64

Mengubah value dari cicil

1 = menyicil

0 = cash

## Mengisi Missing Value

In [ ]:
# memberi value 0 untuk yang tidak menyicil
imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=0)
df[label] = imputer.fit_transform(df[label])
df[label].head(5)

,CICIL
0,0.0
1,0.0
2,0.0
3,1300000.0
4,0.0


In [ ]:
# mengubah tipe data menjadi float
df[label] = df[label].astype(np.float64)
df[label].sum()

CICIL    4.416849e+10
dtype: float64

In [ ]:
# memberi value 1 untuk yang menyicil
for i in df['CICIL']:
    if i != 0: # Jika value tidak 0, mengganti dengan nilai 1
        df['CICIL'].replace(i, 1, inplace = True)

In [ ]:
df[label].head(5)

,CICIL
0,0.0
1,0.0
2,0.0
3,1.0
4,0.0


## Modelling

Menggunakan SVM, KNN, Random Forest, Naive Bayes, dan Logistic Regression

In [ ]:
X = df[used_feature]
y = df[label]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# SVM
clf_svc = SVC(C=1, kernel='linear', gamma='scale')
clf_svc.fit(X_train, y_train)

# prediction result
y_pred = clf_svc.predict(X_test)
print(accuracy_score(y_test,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5778732545649838


In [ ]:
# KNeighborsClassifier
clf_knn=KNeighborsClassifier(n_neighbors=2, algorithm='auto',weights='uniform', leaf_size=30)
clf_knn.fit(X_train, y_train)

# prediction result
y_pred = clf_knn.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.6769602577873255


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Random Forest
clf_rf=RandomForestClassifier(n_estimators=5, criterion='entropy', random_state=42, max_depth=6)
clf_rf.fit(X_train, y_train)

# prediction result
y_pred = clf_rf.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.6162728249194415


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Naive Bayes
clf_nb=GaussianNB()
clf_nb.fit(X_train, y_train)

# prediction result
y_pred = clf_nb.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.585123523093448


/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Logistic Regression
clf_lr=LogisticRegression(C=2, solver='liblinear', multi_class='auto')
clf_lr.fit(X_train, y_train)

# prediction result
y_pred = clf_lr.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.5714285714285714


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Terlihat Akurasi tertinggi didapatkan oleh KNN sebesar 67%

## Save Model

In [ ]:
import pickle
filename = 'konsumen.sav'
pickle.dump(clf_knn, open(filename, 'wb'))

In [ ]:
df.groupby('CICIL').sum()

,TYPE MOTOR,JENIS KELAMIN,STATUS RUMAH,BESAR CICILAN,LAMA CICILAN,PEKERJAAN,PENGELUARAN,PENDIDIKAN,SMH DIGUNAKAN UNTUK,YG MENGGUNAKAN SMH,HOBI
CICIL,,,,,,,,,,,
0.0,130966,3497,6229,0.000000e+00,0.0,71954,24571,28170,21261,21836,151209
1.0,184077,4734,8832,1.050708e+10,322026.0,100750,33914,41851,29943,31526,194149
